### Getting started

**Pure component chemical properties.**

Chemical objects contain pure component properties:

In [1]:
import thermosteam as tmo
# Initialize with an identifier (e.g. name, CAS, InChI...)
water = tmo.Chemical('Water') 
water # All attributes of a chemical are presented in the IPython display

Chemical: Water (phase_ref='l')
[Names]  CAS: 7732-18-5
         InChI: H2O/h1H2
         InChI_key: XLYOFNOQVPJJNP-U...
         common_name: water
         iupac_name: oxidane
         pubchemid: 962
         smiles: O
[Groups] Dortmund: <1H2O>
         UNIFAC: <1H2O>
         PSRK: <1H2O>
[Thermo] S_excess(phase, T, P) -> J/mol
         H_excess(phase, T, P) -> J/mol
         mu(phase, T, P) -> Pa*s
         kappa(phase, T, P) -> W/m/K
         V(phase, T, P) -> m^3/mol
         S(phase, T, P) -> J/mol
         H(phase, T) -> J/mol
         Cn(phase, T) -> J/mol/K
         Psat(T, P=None) -> Pa
         Hvap(T, P=None) -> J/mol
         sigma(T, P=None) -> N/m
         epsilon(T, P=None)
[Data]   MW: 18.015 g/mol
         Tm: 273.15 K
         Tb: 373.12 K
         Tt: 273.15 K
         Tc: 647.14 K
         Pc: 2.2048e+07 Pa
         Vc: 5.6e-05 m^3/mol
         Zc: 0.22947
         Hf: -2.4182e+05 J/mol
         Hc: 0 J/mol
         Hfus: 6010 J/mol
         omega: 0.344
         

The names are strings of chemical identifiers:

In [2]:
water.CAS

'7732-18-5'

The groups are dictionaries of functional groups identifiers for the estimation of activity coefficients through group contribution methods:

In [3]:
water.Dortmund

<DortmundGroupCounts: 1H2O>

The keys are group designation numbers and the values are the number of each group:

In [4]:
dict(water.Dortmund)

{16: 1}

Temperature (in Kelvin) and pressure (in Pascal) dependent properties can be computed through the functors:

In [5]:
# Calculate vapor pressure (Pa)
water.Psat(T=373.15)

101284.55179999319

In [6]:
# Calculate surface tension (N/m)
water.sigma(T=298.15)

0.07205503890847455

In [7]:
# Calculate molar volume (m^3/mol)
water.V(T=298.15, P=101325, phase='l')

1.687456798143492e-05

In [8]:
# Calculate enthalpy at reference conditions (J/mol; without excess energies)
water.H(T=298.15, phase='l')

0.0

Note that the reference state of all chemicals is 25 $^{\circ}$C and 1 atm:

In [9]:
water.T_ref

298.15

In [10]:
water.P_ref

101325.0

Constant pure component properties are also available:

In [11]:
# Molecular weight (g/mol)
water.MW

18.01528

In [12]:
# Boiling point (K)
water.Tb

373.124

Temperature dependent properties are managed by model handles (a functor):

In [13]:
water.Psat

TDependentModelHandle(T, P=None) -> Psat [Pa]
[0] Wagner_McGraw
[1] Antoine
[2] DIPPR_EQ101
[3] Wagner
[4] Boiling_Critical_Relation
[5] Lee_Kesler
[6] Ambrose_Walton
[7] Sanjari
[8] Edalat


A model handle contains a series of models applicable to a certain domain:

In [17]:
water.Psat[0]

TDependentModel: Wagner_McGraw
 evaluate: Wagner_McGraw(T, P=None) -> Psat [Pa]
 Tmin: 275.00
 Tmax: 647.35


When called, the model handle searches through each model until it finds one with an applicable domain. If none are applicable, you get a value error:

In [15]:
water.Psat(1000.0)

ValueError: <TDependentModelHandle(T, P=None) -> Psat [Pa]> contains no valid model at T=1000.00 K

Each model contains a functor that stores data to compute the property:

In [ ]:
water.Psat[0].evaluate

In [ ]:
water.Psat[0].evaluate.data

In [ ]:
water.Psat[0].evaluate.function

In [ ]:
# For further help...
help(water.Psat[0].evaluate.function)

Phase dependent properties have attributes with model handles for each phase:

In [ ]:
water.V

In [ ]:
water.V.l

Note: It is not required to use these functors. They can be replaced by your own function or even a constant as shown in the next section.

**Managing pure component thermodynamic models**

The models are ordered in a [deque](https://docs.python.org/3.6/library/collections.html#collections.deque) object to allow for easy reordering:

In [ ]:
water.Psat.models

In [ ]:
water.Psat.models.rotate(-1)
water.Psat.models

You could also insert your own model:

In [ ]:
@water.Psat.model(Tmin=273.20, Tmax=473.20, top=True) # top=True to place in position 0
def user_antoine_model(T):
    return 10.0**(10.116 -  1687.537 / (T + 42.98))

water.Psat

In [ ]:
You can also set a constant model:
water.V.l(298.15, )

In the next example, we make an ideal mixture property package from chemicals, some of which contain constant values fo

In [ ]:
water =  ether.Chemical('Water')
ethanol = ether.Chemical('Ethanol')
methanol = ether.Chemical('Methanol')
lignin = ether.Chemical('Lignin')
ideal_mixture = ether.IdealMixture(chemicals=(water, ethanol, methanol, lignin))
ideal_mixture

In [ ]:
ideal_mixture.H

In [ ]:
ideal_mixture.H('l', [0.1, 0.2, 0.3, 0.4], 350)